In [1]:
import sys
print(sys.executable)



c:\Users\harsh\Desktop\WEB Wiz\Project\.venv\Scripts\python.exe


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    'spam.csv',
    header=None,
    encoding='latin-1'
    
    )

df[['label', 'text']] = df[0].str.split('\t', expand=True)
df = df[['text', 'label']]

df.head()


,text,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [3]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [4]:
df.duplicated().sum()

np.int64(414)

Data Processessing..

In [5]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def clean_text(text):
    text = text.lower()

    text = re.sub(f"[{string.punctuation}]", " ", text)
    
    text = re.sub(r'\d+', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
    
    words = text.split()
    clean_words = []

    for word in words:
        if word not in stop:
            clean_words.append(word)

    text = " ".join(clean_words)
    return text


df['cleaned'] = df['text'].apply(clean_text)        

df.head()

,text,label,cleaned
0,"Go until jurong point, crazy.. Available only ...",ham,go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry wkly comp win fa cup final tkts st ...
3,U dun say so early hor... U c already then say...,ham,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah think goes usf lives around though


In [7]:
df['target'] = df['label'].map({'ham': 0, 'spam': 1})

df.head()

,text,label,cleaned,target
0,"Go until jurong point, crazy.. Available only ...",ham,go jurong point crazy available bugis n great ...,0
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry wkly comp win fa cup final tkts st ...,1
3,U dun say so early hor... U c already then say...,ham,u dun say early hor u c already say,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah think goes usf lives around though,0


Feature...

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

X = df['cleaned']
Y = df['target']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)


print(f"Training on {len(X_train)} emails")
print(f"Testing on {len(X_test)} emails")

Training on 4459 emails
Testing on 1115 emails


Training using Naive-Bayes Classifier

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline_1 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=3000)),
    ('nb', MultinomialNB())
])

pipeline_1.fit(X_train, Y_train)

Y_pred = pipeline_1.predict(X_test)

print(f"Accuracy Score: {accuracy_score(Y_test, Y_pred)}")
print("\nClassification Report:\n")
print(classification_report(Y_test, Y_pred))

Accuracy Score: 0.9766816143497757

Classification Report:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.98      0.84      0.91       149

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



Prediction

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

pipeline_2 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=3000)),
    ('KNN', KNeighborsClassifier())
])

pipeline_2.fit(X_train, Y_train)

Y_pred = pipeline_2.predict(X_test)

print(f"Accuracy Score: {accuracy_score(Y_test, Y_pred)}")
print("\nClassification Report:\n")
print(classification_report(Y_test, Y_pred))

Accuracy Score: 0.9210762331838565

Classification Report:

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       966
           1       0.96      0.43      0.59       149

    accuracy                           0.92      1115
   macro avg       0.94      0.71      0.77      1115
weighted avg       0.92      0.92      0.91      1115



In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

pipeline_3 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=3000)),
    ('Random Forest', RandomForestClassifier())
])

pipeline_3.fit(X_train, Y_train)

Y_pred = pipeline_3.predict(X_test)

print(f"Accuracy Score: {accuracy_score(Y_test, Y_pred)}")
print("\nClassification Report:\n")
print(classification_report(Y_test, Y_pred))

Accuracy Score: 0.979372197309417

Classification Report:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.85      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [ ]:
def predict_message(message):

    prediction = pipeline_3.predict([message])
    
    if prediction[0] == 1:
        return "SPAM "
    else:
        return "NOT SPAM "


print("Enter your message [type 'exit' to stop]:\n")

while True:
   
    user = input("Your Message: ")
    
    if user.lower() == 'exit':
        print("Exited")
        break
    
    result = predict_message(user)
    print(f"Prediction: {result}\n")